In [1]:
import tensorflow as tf
# use keras.datasets instead of tensorflow.datasets
from tensorflow.keras import datasets, layers
mnist = datasets.mnist

learning_rate = 1e-4
keep_prob_rate = 0.7
max_epoch = 5

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = layers.Conv2D(32, 7, activation='relu')
        self.pool1 = layers.MaxPooling2D()
        self.conv2 = layers.Conv2D(64, 5, activation='relu')
        self.pool2 = layers.MaxPooling2D()
        self.flatten = layers.Flatten()
        self.d1 = layers.Dense(1024, activation='relu')
        self.dropout = layers.Dropout(1-keep_prob_rate)
        self.d2 = layers.Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.dropout(x)
        return self.d2(x)

model = MyModel()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

for epoch in range(max_epoch):
    for i, (images, labels) in enumerate(train_ds):
        train_step(images, labels)
        if i % 100 == 0:
            for test_images, test_labels in test_ds:
                test_step(test_images, test_labels)
            template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
            print(template.format(epoch+1,
                                  train_loss.result(),
                                  train_accuracy.result()*100,
                                  test_loss.result(),
                                  test_accuracy.result()*100))

Epoch 1, Loss: 2.3032445907592773, Accuracy: 21.875, Test Loss: 2.3064169883728027, Test Accuracy: 11.369999885559082
Epoch 1, Loss: 1.7134983539581299, Accuracy: 54.95049285888672, Test Loss: 1.5656007528305054, Test Accuracy: 45.75
Epoch 1, Loss: 1.1362444162368774, Accuracy: 69.77611541748047, Test Loss: 1.1905158758163452, Test Accuracy: 59.59666442871094
Epoch 1, Loss: 0.8882629871368408, Accuracy: 76.00706481933594, Test Loss: 0.975449800491333, Test Accuracy: 67.31749725341797
Epoch 1, Loss: 0.7507299184799194, Accuracy: 79.48877716064453, Test Loss: 0.8346531391143799, Test Accuracy: 72.2959976196289
Epoch 1, Loss: 0.6558398008346558, Accuracy: 81.92989349365234, Test Loss: 0.7382892370223999, Test Accuracy: 75.65167236328125
Epoch 1, Loss: 0.5885500907897949, Accuracy: 83.71463775634766, Test Loss: 0.6642132997512817, Test Accuracy: 78.22000122070312
Epoch 1, Loss: 0.5379213094711304, Accuracy: 85.08826446533203, Test Loss: 0.6065375804901123, Test Accuracy: 80.1937484741211
E